In [21]:
%load_ext autoreload
%autoreload 2
from utils import feature_eng, visualization
# plot the reward function with plotly as line plot
import plotly.io as pio
import numpy as np
import plotly.graph_objs as go
import pmdarima as pm
from pmdarima.model_selection import train_test_split
import numpy as np
import matplotlib.pyplot as plt
from plotly.subplots import make_subplots
import pandas as pd

from utils import eval

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
pio.renderers.default = 'iframe'

measurements = feature_eng.get_data_df("train", "B")
fig = visualization.plot_substation_measurements(measurements, "B")
fig.show()

In [3]:
fig = visualization.plot_production(measurements, ['ZLQZLQSPHSDL_VNUUIST'])
fig.show()

In [4]:
is_data = measurements['ZLQZLQSPHSDL_VNUUIST']
target_data = measurements['ZLQZLQSPHSDL_VNUUSOL']
shunt_reactor = measurements['ZLQZLQ110KOMDROQ']
station_b_val = feature_eng.add_temporal_features(feature_eng.get_data_df("val", "B"))

reactor_border = 20

train, test = train_test_split(is_data.values, train_size=len(is_data)-1)

model = pm.ARIMA(order=(2, 1, 2), seasonal_order=(1, 0, 1, 4))
print("fertig")


fertig


In [8]:
# measurement points are separated by 15 minutes. 4 to get to one hour.
time = 4
loops = 1

predictions = []
train, test = train_test_split(is_data.values, train_size=len(is_data)-1)
print(len(train))
print(len(pd.DataFrame(shunt_reactor[:-1])))
model_fit = model.fit(y=train, X=pd.DataFrame(shunt_reactor[:-1]))
print("fertig")

34940
34940


C:\_dev\edh2023\.venv\Lib\site-packages\statsmodels\base\model.py:607: ConvergenceWarning:

Maximum Likelihood optimization failed to converge. Check mle_retvals



fertig


In [85]:
shunt_reactor_decisions = station_b_val['ZLQZLQ110KOMDROQ']

number_of_switches = 0

#for i in range(len(station_b_val)-1):
for i in range(len(shunt_reactor_decisions)):
    """
    if i == 0:
        model_fit.update(y=[train[i]], X=pd.DataFrame([shunt_reactor[-1]]))
    else:
        model_fit.update(y=[train[i]], X=pd.DataFrame([shunt_reactor_decisions[i-1]]))
    
    # Simulate with shunt reactor running
    preds_reactor_on = model_fit.predict(time, pd.DataFrame(np.full(time, 90)))

    # Simulate with shunt reactor off
    preds_reactor_off = model_fit.predict(time, pd.DataFrame(np.full(time, 0)))
    print(shunt_reactor_decisions[i])
    print(target_data[i])
    print(train[i])
    """

    if i == 0:
        previous_state = shunt_reactor[-1]
    else:
        previous_state = shunt_reactor_decisions[i-1]
        
    if i == 0 or (number_of_switches + 1) / (i*4*24) < 2:
        if train[i] - target_data[i] > 2:
            # current voltage is too high -> start shunt reactor
            shunt_reactor_decisions[i] = 90
            if previous_state < 1:
                number_of_switches += 1
        elif train[i] - target_data[i] < -2:
            # current voltage is too low -> stop shunt reasctor
            shunt_reactor_decisions[i] = 0
            if previous_state > 1:
                number_of_switches += 1
        else:
            shunt_reactor_decisions[i] = shunt_reactor_decisions[i-1]
    elif i != 0:
        shunt_reactor_decisions[i] = shunt_reactor_decisions[i-1]
    #for j in range(3):
        #shunt_reactor_decisions[i+j+1] = shunt_reactor_decisions[i]

print(number_of_switches)

28


In [86]:
last_element = is_data.index[-1]
predicted_data_on = pd.Series(preds_reactor_on, index=pd.date_range(last_element, periods=(time), freq='15T'))
predicted_data_off = pd.Series(preds_reactor_off, index=pd.date_range(last_element, periods=(time), freq='15T'))

fig1 = visualization.plot_substation_measurements(station_b_val, "B")
fig1.show()

# calculate reward points achieved for the validation dataset
reward_base = eval.reward_sum(
    station_b_val["ZLQZLQSPHSDL_VNUUIST"].values - station_b_val["ZLQZLQSPHSDL_VNUUSOL"].values, 
    (station_b_val['ZLQZLQ110KOMDROQ'] > 1).values, 
    weighing_factors=np.ones(station_b_val.shape[0]))
print(f'Reward achieved by operating the shunt reactor as in the validation dataset: {reward_base}.')

scatters = []
scatters.append(go.Scatter(x=is_data.index, y=is_data, name="IST"))

for i in range(loops):
    scatters.append(go.Scatter(x=predicted_data_on.index, y=predicted_data_on, name="PREDICT ON "+str(i)))
    scatters.append(go.Scatter(x=predicted_data_off.index, y=predicted_data_off, name="PREDICT OFF "+str(i)))

fig = go.Figure(data=scatters)

# Update the layout
fig.update_layout(title="Nice", xaxis_title='Date', yaxis_title='kV')
fig.show()

Reward achieved by operating the shunt reactor as in the validation dataset: 22004.4.
